<a href="https://colab.research.google.com/github/lalesafarzade/Recommendation_system_Project/blob/lale/Notebooks/2.Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 5s (50.1 kB/s)
Reading package li

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Apache Spark Libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Movie").config("spark.executor.memory", "1G").config("spark.executor.cores","4").getOrCreate()

In [5]:
url = '/content/drive/MyDrive/movies.csv'

movie = spark.read.format("csv") \
       .option("header", "true") \
       .option("inferSchema","true")\
       .load(url) 

movie.cache()

DataFrame[movieId: int, title: string, genres: string]

In [6]:
#movie=movie.drop('genres')
movie.show(2)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows



In [7]:
url = '/content/drive/MyDrive/ratings.csv'

rating = spark.read.format("csv") \
       .option("header", "true") \
       .option("inferSchema","true")\
       .load(url) 

rating.cache() #for faster re-use

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [8]:
rating.show(2)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
+------+-------+------+----------+
only showing top 2 rows



In [9]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
rating.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in rating.columns]
   ).show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     0|      0|     0|        0|
+------+-------+------+---------+



In [11]:
movie=movie.drop('genres')

In [12]:
df = rating.join(movie,['movieId'],how='inner').distinct()
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- title: string (nullable = true)



In [34]:
df=df.drop('timestamp')
df.show(2)

+-------+------+------+--------------------+
|movieId|userId|rating|               title|
+-------+------+------+--------------------+
|    500| 13424|   3.0|Mrs. Doubtfire (1...|
|      9| 25197|   5.0| Sudden Death (1995)|
+-------+------+------+--------------------+
only showing top 2 rows



In [35]:
(training, test) = df.randomSplit([0.75, 0.25])

In [17]:
rmses=[]
for reg in [.01,0.5,.1,.15]:
  
  als = ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop", nonnegative = True, implicitPrefs = False,regParam=reg)
  model = als.fit(training)
  predictions = model.transform(test)
  evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
  rmses.append(evaluator.evaluate(predictions))



In [20]:
rmses

[0.8070268456936307,
 0.9980713309798638,
 0.8136641375108292,
 0.8369208851400409]

In [23]:
ranks_rmse=[]
for rank in range(10,20):
  
  als = ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop", nonnegative = True, implicitPrefs = False,regParam=.01,rank=rank)
  model = als.fit(training)
  predictions = model.transform(test)
  evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
  ranks_rmse.append(evaluator.evaluate(predictions))

In [24]:
ranks_rmse

[0.8070268456936307,
 0.8044669593529069,
 0.8063713814109872,
 0.8072200779995645,
 0.8082110898664787,
 0.8073313613300547,
 0.8063300764454617,
 0.8083782773740316,
 0.8102906451829985,
 0.8079662524192257]

In [36]:
reg=.01
rank=11
als = ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop", nonnegative = True, implicitPrefs = False,rank=rank,regParam=reg)
model = als.fit(training)

In [37]:
predictions = model.transform(test)

In [38]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8057867401376125


In [45]:
#als = ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop", nonnegative = True, implicitPrefs = False)
#param_grid = ParamGridBuilder().addGrid(als.rank, [10, 50]) .addGrid(als.maxIter, [5,75]) .addGrid(als.regParam, [.05, .15]).build()

# Define evaluator as RMSE
#evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
# Print length of evaluator
#print ("Number of models by using param_grid: ", len(param_grid))
#cv=CrossValidator(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator,numFolds=4)
#cvModel=cv.fit(training)

In [39]:
import tempfile
path1 = tempfile.mkdtemp()
path = path1 + F"/content/drive/MyDrive/spark_model"
model.write().save(path)

In [40]:
path

'/tmp/tmpm61cmxkl/content/drive/MyDrive/spark_model'

In [41]:
loaded_model=model.read().load(path)

In [42]:
predictions.show(n = 20)

+-------+------+------+--------------------+----------+
|movieId|userId|rating|               title|prediction|
+-------+------+------+--------------------+----------+
|   7323|     1|   3.5|Good bye, Lenin! ...|  4.067205|
|   7365|     1|   4.0|Noi the Albino (N...| 3.5133626|
|   8360|     1|   4.0|      Shrek 2 (2004)| 3.0715728|
|   8685|     1|   1.0|Miracle of Marcel...| 1.9768935|
|   8973|     1|   4.0|Bad Education (La...|  4.278549|
|   4703|     1|   4.0|     Chocolat (1988)| 3.4385736|
|   7940|     1|   4.5| The Magician (1958)|  3.572158|
|   5878|     1|   4.0|Talk to Her (Habl...|  4.130054|
|   6954|     1|   3.5|Barbarian Invasio...| 3.9745429|
|   3448|     1|   4.0|Good Morning, Vie...| 3.1550221|
|   5952|     1|   4.0|Lord of the Rings...|  4.488222|
|   2573|     1|   4.0|        Tango (1998)| 2.5541518|
|   4308|     1|   3.0| Moulin Rouge (2001)| 4.3651443|
|   5767|     1|   5.0|Teddy Bear (Mis) ...| 3.8262622|
|   8154|     1|   5.0|Dolce Vita, La (1...|  3.

In [43]:
specific_user_preds = spark.createDataFrame([(7000,3),(7000,4),( 7000,7)], [ "userId","movieId"])
specific_user_preds.show()

+------+-------+
|userId|movieId|
+------+-------+
|  7000|      3|
|  7000|      4|
|  7000|      7|
+------+-------+



In [44]:
new_predictions=loaded_model.transform(specific_user_preds)
new_predictions.show()

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|  7000|      3| 3.9101179|
|  7000|      4| 2.8711562|
|  7000|      7| 3.8532453|
+------+-------+----------+

